In [1]:
using OceananigansLagrangianFilter
using Printf
using JLD2
using Oceananigans.Units: Time
using CairoMakie
using CUDA
using Oceananigans.TimeSteppers: reset!
using GeoStats: IDW


In [2]:
# User defined options

original_data_filename = joinpath(@__DIR__, "SW_vort_new.jld2")
T_start = 0
T_end = 20

arch = GPU()

# Set the output period
T_out = 0.1

# Set filter order and cut-off frequency
# Amplitude of frequency response of filter will be squared Butterworth order 2^N
N = 2 
freq_c = 2

# Define variables to filter
original_var_names = ("ω",)

# Define velocities to use for filtering
velocity_names = ("u","v")

# Set filtering parameters (this is for Butterworth-type, could define others here)
filter_params = set_BW_filter_params(N=N,freq_c=freq_c)

# Set the time step for the simulation
Δt = 1e-3

# Decide whether to solve for and output maps to generalised Lagrangian mean
map_to_mean = true

# Name output files
forward_output_filename = joinpath(@__DIR__, "forward_LF.jld2")
backward_output_filename = joinpath(@__DIR__, "backward_LF.jld2")
combined_output_filename = joinpath(@__DIR__, "combined_LF.jld2")

# Manipulate data on disk to have correct order 
T = set_data_on_disk!(original_data_filename, direction="forward", T_start = T_start, T_end = T_end)

Current direction is forward
No need to reverse order of data


20

In [3]:
# Load in saved data from simulation
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))
println("Loaded data from $original_data_filename")

# Create the original variables - these will be auxiliary fields in the model
original_vars = create_original_vars(original_var_names, grid)
println("Created original variables: ", original_vars)

# Create the filtered variables - these will be tracers in the model
filtered_vars = create_filtered_vars(original_var_names, velocity_names, filter_params, map_to_mean=map_to_mean)
println("Created filtered variables: ", filtered_vars)

# Create forcing for these filtered variables
forcing = create_forcing(filtered_vars, original_var_names, velocity_names, filter_params)
println("Created forcing for filtered variables ")

# Define model 
model = LagrangianFilter(;grid, tracers = filtered_vars, auxiliary_fields = original_vars, forcing = forcing)
println("Created model")

# Define our outputs # This seems to take a while
filtered_outputs = create_output_fields(model, original_var_names, velocity_names, filter_params)
println("Defined outputs")

# Define the filtering simulation 
simulation = Simulation(model, Δt = Δt, stop_time = T) 
println("Defined simulation")

# Tell the simulation to use the saved data
simulation.callbacks[:update_input_data] = Callback(update_input_data!, parameters = (velocities = saved_velocities, original_vars = saved_original_vars))

# Add a progress monitor
function progress(sim)
    @info @sprintf("Simulation time: %s, max(|u|):%.2e \n", 
                   prettytime(sim.model.clock.time), 
                   maximum(abs, model.velocities.u))             
     return nothing
 end

simulation.callbacks[:progress] = Callback(progress, IterationInterval(50))

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = forward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

run!(simulation)

Loaded data from /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/SW_vort_new.jld2
Created original variables: NamedTuple with 1 Fields on 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo:
└── ω: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
Created filtered variables: (:ωC1, :ωC2, :xi_u_C1, :xi_u_C2, :xi_v_C1, :xi_v_C2, :ωS1, :ωS2, :xi_u_S1, :xi_u_S2, :xi_v_S1, :xi_v_S2)
Created forcing for filtered variables 
Created model
Defined outputs
Defined simulation


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):9.69e-01 
[ Info:     ... simulation initialization complete (33.629 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (10.531 seconds).
[ Info: Simulation time: 50.000 ms, max(|u|):9.55e-01 
[ Info: Simulation time: 100.000 ms, max(|u|):9.92e-01 
[ Info: Simulation time: 148.000 ms, max(|u|):9.70e-01 
[ Info: Simulation time: 198.000 ms, max(|u|):1.01e+00 
[ Info: Simulation time: 248.000 ms, max(|u|):9.73e-01 
[ Info: Simulation time: 298.000 ms, max(|u|):1.01e+00 
[ Info: Simulation time: 348.000 ms, max(|u|):9.83e-01 
[ Info: Simulation time: 398.000 ms, max(|u|):1.01e+00 
[ Info: Simulation time: 448.000 ms, max(|u|):1.02e+00 
[ Info: Simulation time: 498.000 ms, max(|u|):1.07e+00 
[ Info: Simulation time: 548 ms, max(|u|):1.07e+00 
[ Info: Simulation time: 598.000 ms, max(|u|):1.11e+00 
[ Info: Simulation time: 648.000 ms, max(|u|):1.11e+00 
[ Info: Simulation 

In [4]:

# Now, run it backwards. Switch the data direction on disk
T = set_data_on_disk!(original_data_filename, direction="backward", T_start = T_start, T_end = T_end)

# Reload the saved data 
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))

# Reset the filtering simulation 
reset!(model.clock)

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = backward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

# And run the backward simulation.
run!(simulation)


Current direction is forward
Reversing order of data
Reversed order of ω
Reversed order of and switched sign of u
Reversed order of and switched sign of v
Reversed order of t_simulation
Reversed order of and shifted t
New direction is backward


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):1.10e+00 
[ Info:     ... simulation initialization complete (30.394 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (5.454 ms).
[ Info: Simulation time: 50.000 ms, max(|u|):1.00e+00 
[ Info: Simulation time: 100.000 ms, max(|u|):9.69e-01 
[ Info: Simulation time: 148.000 ms, max(|u|):9.09e-01 
[ Info: Simulation time: 198.000 ms, max(|u|):9.48e-01 
[ Info: Simulation time: 248.000 ms, max(|u|):9.39e-01 
[ Info: Simulation time: 298.000 ms, max(|u|):1.03e+00 
[ Info: Simulation time: 348.000 ms, max(|u|):1.01e+00 
[ Info: Simulation time: 398.000 ms, max(|u|):1.08e+00 
[ Info: Simulation time: 448.000 ms, max(|u|):1.06e+00 
[ Info: Simulation time: 498.000 ms, max(|u|):1.12e+00 
[ Info: Simulation time: 548 ms, max(|u|):1.14e+00 
[ Info: Simulation time: 598.000 ms, max(|u|):1.21e+00 
[ Info: Simulation time: 648.000 ms, max(|u|):1.18e+00 
[ Info: Simulation time: 698.0

In [5]:
sum_forward_backward_contributions!(combined_output_filename, forward_output_filename, backward_output_filename, T, velocity_names, original_var_names)

Combined forward and backward contributions into /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/combined_LF.jld2


In [6]:
# Next step is to interpolate onto the correct grid
regrid_to_mean_position!(combined_output_filename, original_var_names, velocity_names, IDW(), 5)



0.0%┣                                              ┫ 0/201 [00:00<00:00, -0s/it]
0.5%┣▏                                         ┫ 1/201 [00:03<Inf:Inf, InfGs/it]
1.0%┣▌                                              ┫ 2/201 [00:04<13:49, 4s/it]
1.5%┣▊                                              ┫ 3/201 [00:05<09:01, 3s/it]
2.0%┣█                                              ┫ 4/201 [00:07<07:25, 2s/it]
2.5%┣█▏                                             ┫ 5/201 [00:08<06:36, 2s/it]
3.0%┣█▍                                             ┫ 6/201 [00:09<06:05, 2s/it]
3.5%┣█▋                                             ┫ 7/201 [00:11<05:45, 2s/it]
4.0%┣█▉                                             ┫ 8/201 [00:12<05:30, 2s/it]
4.5%┣██                                             ┫ 9/201 [00:13<05:18, 2s/it]
5.0%┣██▎                                           ┫ 10/201 [00:15<05:09, 2s/it]
5.5%┣██▌                                           ┫ 11/201 [00:16<05:01, 2s/it]
6.0%┣██▊                    

In [7]:
# Animate
timeseries1 = FieldTimeSeries(combined_output_filename, "ω")
timeseries2 = FieldTimeSeries(combined_output_filename, "ω_filtered")
timeseries3 = FieldTimeSeries(combined_output_filename, "ω_filtered_regrid")
times = timeseries1.times

set_theme!(Theme(fontsize = 20))
fig = Figure(size = (1000, 500))

axis_kwargs = (xlabel = "x",
               ylabel = "y",
               limits = ((0, 2π), (0, 2π)),
               aspect = AxisAspect(1))

ax1 = Axis(fig[2, 1]; title = "Raw vorticity", axis_kwargs...)
ax2 = Axis(fig[2, 2]; title = "Filtered vorticity", axis_kwargs...)
ax3 = Axis(fig[2, 3]; title = "Filtered regridded vorticity", axis_kwargs...)


n = Observable(1)
Observable(1)

var1 = @lift timeseries1[$n]
var2 = @lift timeseries2[$n]
var3 = @lift timeseries3[$n]

heatmap!(ax1, var1; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax2, var2; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax3, var3; colormap = :balance, colorrange = (-1, 1))


title = @lift "t = " * string(round(times[$n], digits=2))
Label(fig[1, 1:3], title, fontsize=24, tellwidth=false)

fig

frames = 1:length(times)

@info "Making an animation"

CairoMakie.record(fig, "LF_movie.mp4", frames, framerate=24) do i
    n[] = i
end

[ Info: Making an animation


"LF_movie.mp4"